In [1]:
import json
import copy
import pathlib

In [2]:
# this needs to be modified for the topology being used
clientPools = ['clientPoolA', 'clientPoolB', 'clientPoolC']
default_node_for_services = 'serverX'
default_node_for_services_region = 'X'

path_to_scenario = pathlib.Path('../src/test/resources/ns2/multinode-lan_ms-trace/')
path_to_trace_data = pathlib.Path('../src/test/resources/ms_trace/subscriber_id/')

input_demand_filename = path_to_trace_data / 'demand/clientPoolA.json'

demand_output = path_to_scenario / 'trace-data_spread-clients'
demand_output.mkdir(parents=True, exist_ok=True)

In [3]:
# load the raw demand
with open(input_demand_filename) as f:
    raw_demand = json.load(f)

In [5]:
# create new service names

service_names = dict()
new_demand = copy.deepcopy(raw_demand)
for d in new_demand:
    service = d['service']
    service_artifact = service['artifact']
    if service_artifact not in service_names:
        name = "service-{}".format(len(service_names))
        service_names[service_artifact] = name
    else:
        name = service_names[service_artifact]
        
    # add CPU load equal to TASK_CONTAINERS load
    d['nodeLoad']['CPU'] = d['nodeLoad']['TASK_CONTAINERS']
        
    service['artifact'] = name
service_names

{'/QYaJTboNn4+BvBkGKTu4pnx4FTpPndls+BBzPuuboX3rr47gaLGhTL7aU7FW5yq': 'service-1',
 '0fpsy/yA9AWhH80hAXtBONOsZ8+groab/oS/Dzmu1OjUL/keqhlPzHwd+meDkaqX': 'service-2',
 '4z3+UUhL6wdvdn87+W68GWASquHG2O03ZuFpK/mrn13Afv+VvF8re/AAaANakM5i': 'service-0',
 'EGcWi04tipE9bKu8mMea9qQyQxXMH2i51C907ohmNx9vCB1JEw9n8m2ks9rpUr1Z': 'service-4',
 'GlHFHm4e5v3aCiiUJdiKFmdrrRhhDQ62E7V9VN0sznI03Af+D/MpaeLEyTDXE6fR': 'service-9',
 'GxkpL8GbUueQE+B/rs8IQId/TF9/JvwaIWuJGFhqEu0BMdXDWzo1GbOGiunD8IAM': 'service-7',
 'MdkYmOa+MyxtoUE5FsCkLby6jbATcdbCOiRWSyQ2jg+tw9ph0QGNGE8IGcp0Mi69': 'service-5',
 'Mx0wcxMu0M7IsF2hDBuyx0ZyL6Ve+GZKw1n0WWkzs/dPUViIj8YBF6CSYenuWLDa': 'service-3',
 'Vk6v+yWTWaNURSYcSqbEs8RgjH6GFPQT71mh7IOs3tWOGX8v+mN35Aoc+cosB2Id': 'service-6',
 'i1uAr/UhnV6UaQWYl3wDKVMV/xWcBd6eKGpaK1WnX1yjL+F8K8YgJ11EC9H7/PrE': 'service-8'}

In [6]:
# spread client demand across all of the client pools
client_demand = dict()

for idx, d in enumerate(new_demand):
    clientIdx = idx % len(clientPools)
    client = clientPools[clientIdx]
    if client not in client_demand:
        client_demand[client] = list()
    client_demand[client].append(d)

for client, clientDemand in client_demand.items():
    filename = demand_output / "{}.json".format(client)
    with open(filename, 'w') as f:
        json.dump(clientDemand, f, sort_keys=True, indent=4)

In [18]:
# create updated service configurations

service_configs = list()
for raw, name in service_names.items():
    service = dict()
    service['service'] = {'group': 'com.bbn', 'artifact': name, 'version': '1'}
    service['hostname'] = name
    service['defaultNode'] = default_node_for_services
    service['defaultNodeRegion'] = default_node_for_services_region
    service['initialInstances'] = '1'
    service['computeCapacity'] = {"TASK_CONTAINERS" : "1"}
    service["networkCapacity"] =  {
      "DATARATE_TX": "100",
      "DATARATE_RX": "100"
    }
    service_configs.append(service)
service_configs

[{'computeCapacity': {'TASK_CONTAINERS': '1'},
  'defaultNode': 'serverX',
  'defaultNodeRegion': 'X',
  'hostname': 'service-0',
  'initialInstances': '1',
  'networkCapacity': {'DATARATE_RX': '100', 'DATARATE_TX': '100'},
  'service': {'artifact': 'service-0', 'group': 'com.bbn', 'version': '1'}},
 {'computeCapacity': {'TASK_CONTAINERS': '1'},
  'defaultNode': 'serverX',
  'defaultNodeRegion': 'X',
  'hostname': 'service-1',
  'initialInstances': '1',
  'networkCapacity': {'DATARATE_RX': '100', 'DATARATE_TX': '100'},
  'service': {'artifact': 'service-1', 'group': 'com.bbn', 'version': '1'}},
 {'computeCapacity': {'TASK_CONTAINERS': '1'},
  'defaultNode': 'serverX',
  'defaultNodeRegion': 'X',
  'hostname': 'service-2',
  'initialInstances': '1',
  'networkCapacity': {'DATARATE_RX': '100', 'DATARATE_TX': '100'},
  'service': {'artifact': 'service-2', 'group': 'com.bbn', 'version': '1'}},
 {'computeCapacity': {'TASK_CONTAINERS': '1'},
  'defaultNode': 'serverX',
  'defaultNodeRegion':

In [19]:
with open(path_to_scenario / 'service-configurations.json', 'w') as f:
    json.dump(service_configs, f, sort_keys=True, indent=4)